## 隔日沖策略 各價位_勝率分析

### 計算各價位的勝率

In [3]:
import pandas as pd
import os
from datetime import datetime, timedelta
from openpyxl.utils import get_column_letter

# --- 設定區 ---
# 存放每日回測結果的資料夾名稱
DATA_DIRECTORY = 'out_oneNight'
# 最終分析報告的存檔名稱
OUTPUT_FILENAME = '1_隔日沖績效分析_各價格區間.xlsx'
# --- 設定結束 ---

def analyze_performance_by_price(data_dir, output_file):
    """
    讀取資料夾中所有的 Excel 回測結果，
    分析各價格區間的勝率，並將結果儲存到指定的 Excel 檔案中。
    """

    # --- 1. 彙總所有歷史交易資料 ---
    all_files = [f for f in os.listdir(data_dir) if f.endswith('_漲幅.xlsx')]
    if not all_files:
        print(f"錯誤：在資料夾 '{data_dir}' 中找不到任何 _漲幅.xlsx 的回測檔案。")
        return

    print(f"找到了 {len(all_files)} 個回測檔案，開始進行彙總分析...")
    
    # 讀取所有 excel 檔並合併成一個大的 DataFrame
    all_trades_df = pd.concat(
        [pd.read_excel(os.path.join(data_dir, f)) for f in all_files],
        ignore_index=True
    )
    
    # 只保留有計算結果的交易
    completed_trades = all_trades_df[all_trades_df['損益%'].notna()].copy()
    if completed_trades.empty:
        print("所有檔案中都沒有已完成的交易紀錄，無法進行分析。")
        return

    # --- 2. 根據「當日收盤價」進行價格區間分類 ---
    # 設定價格區間的分界點
    bins = [0, 50, 100, 200, 300, 400, 500, 1000, float('inf')]
    # 設定每個區間的名稱
    labels = ['0-50元', '50-100元', '100-200元', '200-300元', '300-400元', '400-500元', '500-1000元', '1000元以上']
    
    # 使用 pandas 的 cut 函式，自動為每一筆交易貼上價格區間的標籤
    completed_trades['價格區間'] = pd.cut(
        completed_trades['當日收盤價'], 
        bins=bins, 
        labels=labels, 
        right=False # 包含左邊界，例如 [0, 50)
    )
    
    # --- 3. 計算各價格區間的勝率 ---
    # 建立一個 '是否獲利' 的欄位 (停利=1, 停損=0)
    completed_trades['是否獲利'] = (completed_trades['停利/停損'] == '停利').astype(int)
    
    # # 使用 groupby 進行分組(當日收盤價去分組)，並計算每個組的「交易次數」和「平均獲利率(即勝率)」
    # analysis_result = completed_trades.groupby('價格區間')['是否獲利'].agg(['size', 'mean'])
    
    # # 重新命名欄位，讓報表更清晰
    # analysis_result.rename(columns={'size': '交易次數', 'mean': '勝率%'}, inplace=True)

    # 使用 groupby 進行分組，並計算多個指標
    analysis_result = completed_trades.groupby('價格區間')['損益%'].agg([
        ('交易次數', 'count'),
        ('總勝率%', lambda x: (x > 0).mean()),
        ('平均獲利%', lambda x: x[x > 0].mean()),
        ('平均虧損%', lambda x: x[x < 0].mean())
    ])

    # 重新命名欄位
    analysis_result.rename(columns={'總勝率%': '勝率%'}, inplace=True)

    # 將勝率轉換為百分比格式
    analysis_result['勝率%'] = analysis_result['勝率%'] * 100
    
    # 篩選掉沒有交易紀錄的區間
    final_report = analysis_result[analysis_result['交易次數'] > 0].copy()
    
    # --- 4. 呈現分析結果 ---
    # print("\n--- 隔日沖策略：各價格區間勝率分析 ---")
    # formatted_report_string = final_report.to_string(formatters={'勝率': '{:,.2f}%'.format})
    # print(formatted_report_string)

    # 將分析結果儲存到 Excel 檔案
    try:
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            # 1. 先將分析報告寫入 Excel
            final_report.to_excel(writer, sheet_name='價格區間分析')
            
            # 2. 取得工作表物件，準備進行格式調整
            worksheet = writer.sheets['價格區間分析']
            
            # [final_report.index.name] 會得到 ['價格區間']
            # final_report.columns.to_list() 會得到 ['交易次數', '勝率%']
            # 兩者相加，得到我們要處理的所有標題
            all_headers = [final_report.index.name] + final_report.columns.to_list()
            
            for i, col_name in enumerate(all_headers):
                column_letter = get_column_letter(i + 1)
                
                # 計算內容最大長度 (包含索引)
                if i == 0:
                    max_len = final_report.index.astype(str).map(len).max()
                else:
                    max_len = final_report[col_name].astype(str).map(len).max()
                
                # 比較內容和標題的長度 (考慮中文字元較寬)
                header_length = len(str(col_name).encode('utf-8')) * 0.7 
                adjusted_width = max(header_length, max_len) + 4
                worksheet.column_dimensions[column_letter].width = adjusted_width
            
            # 寫入執行時間戳記
            timestamp_str = f"報告產出時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
            timestamp_row = len(final_report) + 4
            worksheet.cell(row=timestamp_row, column=1, value=timestamp_str)
        
        # final_report.to_excel(output_file)
        print(f"\n✅ 分析報告已成功儲存至: {output_file}")
    except Exception as e:
        print(f"\n❌ 儲存檔案時發生錯誤: {e}")

if __name__ == "__main__":

    # 使用 os.path.join 組合出完整的輸出檔案路徑
    output_filepath = os.path.join(DATA_DIRECTORY, OUTPUT_FILENAME)
    
    # 將組合好的完整路徑傳入函式
    analyze_performance_by_price(DATA_DIRECTORY, output_filepath)


找到了 13 個回測檔案，開始進行彙總分析...

✅ 分析報告已成功儲存至: out_oneNight\1_隔日沖績效分析_各價格區間.xlsx
